In [9]:
import numpy as np
import tifffile as tiff
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from utils import log

def load_volume(path):
    return tiff.imread(path).astype(np.float32)

def normalize(volume):
    return volume / 65535.0  # normalize 16-bit to [0, 1]

def compare_volumes(gt, method, mask, method_name):
    psnr_list = []
    ssim_list = []
    mae_list = []
    mse_list = []

    for i in range(gt.shape[0]):
        if mask[i].mean() > 0.5:  # Only evaluate missing slices
            gt_slice = gt[i]
            pred_slice = method[i]

            mse_value = np.mean((gt_slice - pred_slice) ** 2)
            mae_value = np.mean(np.abs(gt_slice - pred_slice))

            if mse_value == 0:
                psnr_score = 100.0
            else:
                psnr_score = psnr(gt_slice, pred_slice, data_range=1.0)

            ssim_score = ssim(gt_slice, pred_slice, data_range=1.0)

            psnr_list.append(psnr_score)
            ssim_list.append(ssim_score)
            mae_list.append(mae_value)
            mse_list.append(mse_value)

    avg_psnr = np.mean(psnr_list)
    avg_ssim = np.mean(ssim_list)
    avg_mae = np.mean(mae_list)
    avg_mse = np.mean(mse_list)

    log(f"{method_name}:")
    log(f"  Mean PSNR (missing slices only): {avg_psnr:.2f} dB")
    log(f"  Mean SSIM (missing slices only): {avg_ssim:.4f}")
    log(f"  Mean MAE  (missing slices only): {avg_mae:.6f}")
    log(f"  Mean MSE  (missing slices only): {avg_mse:.6f}")
    log("-" * 40)

    return psnr_list, ssim_list, mae_list, mse_list

def run_comparison(gt_path, predicted_path_1, predicted_path_2, mask_path):
    # Load volumes
    gt = normalize(load_volume(gt_path))
    pred1 = normalize(load_volume(predicted_path_1))
    pred2 = normalize(load_volume(predicted_path_2))
    mask = tiff.imread(mask_path)  # mask expected as uint8 or uint16 0/1 format

    # Ensure shapes match
    assert gt.shape == pred1.shape == pred2.shape == mask.shape, "Volumes and mask must match in shape!"

    # Compare volumes
    compare_volumes(gt, pred1, mask, "Predicted Volume 1")
    compare_volumes(gt, pred2, mask, "Predicted Volume 2")


In [ ]:
run_comparison(
    gt_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_gt.tif",
    predicted_path_1="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_corrupted_LinearInterp.tif",
    predicted_path_2="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_inpainted_2p5DUNet_v4_brightcorr.tif",
    mask_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_mask.tif"
)

Linear Interpolation:
  Mean PSNR (missing slices only): 34.78 dB
  Mean SSIM (missing slices only): 0.7956
  Mean MAE  (missing slices only): 0.012557
  Mean MSE  (missing slices only): 0.000335
----------------------------------------
2.5D CNN Prediction:
  Mean PSNR (missing slices only): 36.03 dB
  Mean SSIM (missing slices only): 0.8457
  Mean MAE  (missing slices only): 0.010603
  Mean MSE  (missing slices only): 0.000252
----------------------------------------


In [10]:
run_comparison(
    gt_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.1_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_gt.tif",
    predicted_path_1="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.1_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_inpainted_2p5DUNet_fold4_brightcorr.tif",
    predicted_path_2="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.1_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_inpainted_2p5DUNet_fold4_v2_brightcorr.tif",
    mask_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.1_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_mask.tif"
)

run_comparison(
    gt_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.2_OCT_uint16_Cropped_VolumeSplit_2_Reflected_RegSeq_seqSVD_gt.tif",
    predicted_path_1="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.2_OCT_uint16_Cropped_VolumeSplit_2_Reflected_RegSeq_seqSVD_inpainted_2p5DUNet_fold1_brightcorr.tif",
    predicted_path_2="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.2_OCT_uint16_Cropped_VolumeSplit_2_Reflected_RegSeq_seqSVD_inpainted_2p5DUNet_fold1_v2_brightcorr.tif",
    mask_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.2_OCT_uint16_Cropped_VolumeSplit_2_Reflected_RegSeq_seqSVD_mask.tif"
)

run_comparison(
    gt_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.4_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_gt.tif",
    predicted_path_1="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.4_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_inpainted_2p5DUNet_fold3_brightcorr.tif",
    predicted_path_2="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.4_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_inpainted_2p5DUNet_fold3_v2_brightcorr.tif",
    mask_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.4_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_mask.tif"
)

run_comparison(
    gt_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_gt.tif",
    predicted_path_1="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_inpainted_2p5DUNet_fold5_brightcorr.tif",
    predicted_path_2="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_inpainted_2p5DUNet_fold5_v2_brightcorr.tif",
    mask_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_mask.tif"
)

run_comparison(
    gt_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_gt.tif",
    predicted_path_1="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_inpainted_2p5DUNet_fold2_brightcorr.tif",
    predicted_path_2="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_inpainted_2p5DUNet_fold2_v2_brightcorr.tif",
    mask_path="/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_mask.tif"
)

Predicted Volume 1:
  Mean PSNR (missing slices only): 35.28 dB
  Mean SSIM (missing slices only): 0.8244
  Mean MAE  (missing slices only): 0.011313
  Mean MSE  (missing slices only): 0.000300
----------------------------------------
Predicted Volume 2:
  Mean PSNR (missing slices only): 35.30 dB
  Mean SSIM (missing slices only): 0.8245
  Mean MAE  (missing slices only): 0.011304
  Mean MSE  (missing slices only): 0.000299
----------------------------------------
Predicted Volume 1:
  Mean PSNR (missing slices only): 35.50 dB
  Mean SSIM (missing slices only): 0.8262
  Mean MAE  (missing slices only): 0.011467
  Mean MSE  (missing slices only): 0.000285
----------------------------------------
Predicted Volume 2:
  Mean PSNR (missing slices only): 35.50 dB
  Mean SSIM (missing slices only): 0.8262
  Mean MAE  (missing slices only): 0.011456
  Mean MSE  (missing slices only): 0.000285
----------------------------------------
Predicted Volume 1:
  Mean PSNR (missing slices only): 36.64